# Building your own algorithm container

With Amazon SageMaker, you can package your own algorithms that can than be trained and deployed in the SageMaker environment. This notebook will guide you through an example that shows you how to build a Docker container for SageMaker and use it for training and inference.

By packaging an algorithm in a container, you can bring almost any code to the Amazon SageMaker environment, regardless of programming language, environment, framework, or dependencies. 

1. [Building your own algorithm container](#Building-your-own-algorithm-container)
  1. [When should I build my own algorithm container?](#When-should-I-build-my-own-algorithm-container?)
  1. [Permissions](#Permissions)
  1. [The example](#The-example)
  1. [The presentation](#The-presentation)
1. [Part 1: Packaging and Uploading your Algorithm for use with Amazon SageMaker](#Part-1:-Packaging-and-Uploading-your-Algorithm-for-use-with-Amazon-SageMaker)
    1. [An overview of Docker](#An-overview-of-Docker)
    1. [How Amazon SageMaker runs your Docker container](#How-Amazon-SageMaker-runs-your-Docker-container)
      1. [Running your container during training](#Running-your-container-during-training)
        1. [The input](#The-input)
        1. [The output](#The-output)
      1. [Running your container during hosting](#Running-your-container-during-hosting)
    1. [The parts of the sample container](#The-parts-of-the-sample-container)
    1. [The Dockerfile](#The-Dockerfile)
    1. [Building and registering the container](#Building-and-registering-the-container)
  1. [Testing your algorithm on your local machine or on an Amazon SageMaker notebook instance](#Testing-your-algorithm-on-your-local-machine-or-on-an-Amazon-SageMaker-notebook-instance)
1. [Part 2: Training and Hosting your Algorithm in Amazon SageMaker](#Part-2:-Training-and-Hosting-your-Algorithm-in-Amazon-SageMaker)
  1. [Set up the environment](#Set-up-the-environment)
  1. [Create the session](#Create-the-session)
  1. [Upload the data for training](#Upload-the-data-for-training)
  1. [Create an estimator and fit the model](#Create-an-estimator-and-fit-the-model)
  1. [Deploy the model](#Deploy-the-model)
  1. [Choose some data and use it for a prediction](#Choose-some-data-and-use-it-for-a-prediction)
  1. [Optional cleanup](#Optional-cleanup)  

_or_ I'm impatient, just [let me see the code](#The-Dockerfile)!

## When should I build my own algorithm container?

You may not need to create a container to bring your own code to Amazon SageMaker. When you are using a framework (such as Apache MXNet or TensorFlow) that has direct support in SageMaker, you can simply supply the Python code that implements your algorithm using the SDK entry points for that framework. This set of frameworks is continually expanding, so we recommend that you check the current list if your algorithm is written in a common machine learning environment.

Even if there is direct SDK support for your environment or framework, you may find it more effective to build your own container. If the code that implements your algorithm is quite complex on its own or you need special additions to the framework, building your own container may be the right choice.

If there isn't direct SDK support for your environment, don't worry. You'll see in this walk-through that building your own container is quite straightforward.

## Permissions

Running this notebook requires permissions in addition to the normal `SageMakerFullAccess` permissions. This is because we'll creating new repositories in Amazon ECR. The easiest way to add these permissions is simply to add the managed policy `AmazonEC2ContainerRegistryFullAccess` to the role that you used to start your notebook instance. There's no need to restart your notebook instance when you do this, the new permissions will be available immediately.

## The example

Here, we'll show how to package a simple Python example which showcases the [decision tree][] algorithm from the widely used [scikit-learn][] machine learning package. The example is purposefully fairly trivial since the point is to show the surrounding structure that you'll want to add to your own code so you can train and host it in Amazon SageMaker.

The ideas shown here will work in any language or environment. You'll need to choose the right tools for your environment to serve HTTP requests for inference, but good HTTP environments are available in every language these days.

In this example, we use a single image to support training and hosting. This is easy because it means that we only need to manage one image and we can set it up to do everything. Sometimes you'll want separate images for training and hosting because they have different requirements. Just separate the parts discussed below into separate Dockerfiles and build two images. Choosing whether to have a single image or two images is really a matter of which is more convenient for you to develop and manage.

If you're only using Amazon SageMaker for training or hosting, but not both, there is no need to build the unused functionality into your container.

[scikit-learn]: http://scikit-learn.org/stable/
[decision tree]: http://scikit-learn.org/stable/modules/tree.html

## The presentation

This presentation is divided into two parts: _building_ the container and _using_ the container.

# Part 1: Packaging and Uploading your Algorithm for use with Amazon SageMaker

### An overview of Docker

If you're familiar with Docker already, you can skip ahead to the next section.

For many data scientists, Docker containers are a new concept, but they are not difficult, as you'll see here. 

Docker provides a simple way to package arbitrary code into an _image_ that is totally self-contained. Once you have an image, you can use Docker to run a _container_ based on that image. Running a container is just like running a program on the machine except that the container creates a fully self-contained environment for the program to run. Containers are isolated from each other and from the host environment, so the way you set up your program is the way it runs, no matter where you run it.

Docker is more powerful than environment managers like conda or virtualenv because (a) it is completely language independent and (b) it comprises your whole operating environment, including startup commands, environment variable, etc.

In some ways, a Docker container is like a virtual machine, but it is much lighter weight. For example, a program running in a container can start in less than a second and many containers can run on the same physical machine or virtual machine instance.

Docker uses a simple file called a `Dockerfile` to specify how the image is assembled. We'll see an example of that below. You can build your Docker images based on Docker images built by yourself or others, which can simplify things quite a bit.

Docker has become very popular in the programming and devops communities for its flexibility and well-defined specification of the code to be run. It is the underpinning of many services built in the past few years, such as [Amazon ECS].

Amazon SageMaker uses Docker to allow users to train and deploy arbitrary algorithms.

In Amazon SageMaker, Docker containers are invoked in a certain way for training and a slightly different way for hosting. The following sections outline how to build containers for the SageMaker environment.

Some helpful links:

* [Docker home page](http://www.docker.com)
* [Getting started with Docker](https://docs.docker.com/get-started/)
* [Dockerfile reference](https://docs.docker.com/engine/reference/builder/)
* [`docker run` reference](https://docs.docker.com/engine/reference/run/)

[Amazon ECS]: https://aws.amazon.com/ecs/

### How Amazon SageMaker runs your Docker container

Because you can run the same image in training or hosting, Amazon SageMaker runs your container with the argument `train` or `serve`. How your container processes this argument depends on the container:

* In the example here, we don't define an `ENTRYPOINT` in the Dockerfile so Docker will run the command `train` at training time and `serve` at serving time. In this example, we define these as executable Python scripts, but they could be any program that we want to start in that environment.
* If you specify a program as an `ENTRYPOINT` in the Dockerfile, that program will be run at startup and its first argument will be `train` or `serve`. The program can then look at that argument and decide what to do.
* If you are building separate containers for training and hosting (or building only for one or the other), you can define a program as an `ENTRYPOINT` in the Dockerfile and ignore (or verify) the first argument passed in. 

#### Running your container during training

When Amazon SageMaker runs training, your `train` script is run just like a regular Python program. A number of files are laid out for your use, under the `/opt/ml` directory:

    /opt/ml
    ├── input
    │   ├── config
    │   │   ├── hyperparameters.json
    │   │   └── resourceConfig.json
    │   └── data
    │       └── <channel_name>
    │           └── <input data>
    ├── model
    │   └── <model files>
    └── output
        └── failure

##### The input

* `/opt/ml/input/config` contains information to control how your program runs. `hyperparameters.json` is a JSON-formatted dictionary of hyperparameter names to values. These values will always be strings, so you may need to convert them. `resourceConfig.json` is a JSON-formatted file that describes the network layout used for distributed training. Since scikit-learn doesn't support distributed training, we'll ignore it here.
* `/opt/ml/input/data/<channel_name>/` (for File mode) contains the input data for that channel. The channels are created based on the call to CreateTrainingJob but it's generally important that channels match what the algorithm expects. The files for each channel will be copied from S3 to this directory, preserving the tree structure indicated by the S3 key structure. 
* `/opt/ml/input/data/<channel_name>_<epoch_number>` (for Pipe mode) is the pipe for a given epoch. Epochs start at zero and go up by one each time you read them. There is no limit to the number of epochs that you can run, but you must close each pipe before reading the next epoch.

##### The output

* `/opt/ml/model/` is the directory where you write the model that your algorithm generates. Your model can be in any format that you want. It can be a single file or a whole directory tree. SageMaker will package any files in this directory into a compressed tar archive file. This file will be available at the S3 location returned in the `DescribeTrainingJob` result.
* `/opt/ml/output` is a directory where the algorithm can write a file `failure` that describes why the job failed. The contents of this file will be returned in the `FailureReason` field of the `DescribeTrainingJob` result. For jobs that succeed, there is no reason to write this file as it will be ignored.

#### Running your container during hosting

Hosting has a very different model than training because hosting is reponding to inference requests that come in via HTTP. In this example, we use our recommended Python serving stack to provide robust and scalable serving of inference requests:

![Request serving stack](stack.png)

This stack is implemented in the sample code here and you can mostly just leave it alone. 

Amazon SageMaker uses two URLs in the container:

* `/ping` will receive `GET` requests from the infrastructure. Your program returns 200 if the container is up and accepting requests.
* `/invocations` is the endpoint that receives client inference `POST` requests. The format of the request and the response is up to the algorithm. If the client supplied `ContentType` and `Accept` headers, these will be passed in as well. 

The container will have the model files in the same place they were written during training:

    /opt/ml
    └── model
        └── <model files>



### The parts of the sample container

In the `container` directory are all the components you need to package the sample algorithm for Amazon SageMager:

    .
    ├── Dockerfile
    ├── build_and_push.sh
    └── decision_trees
        ├── nginx.conf
        ├── predictor.py
        ├── serve
        ├── train
        └── wsgi.py

Let's discuss each of these in turn:

* __`Dockerfile`__ describes how to build your Docker container image. More details below.
* __`build_and_push.sh`__ is a script that uses the Dockerfile to build your container images and then pushes it to ECR. We'll invoke the commands directly later in this notebook, but you can just copy and run the script for your own algorithms.
* __`decision_trees`__ is the directory which contains the files that will be installed in the container.
* __`local_test`__ is a directory that shows how to test your new container on any computer that can run Docker, including an Amazon SageMaker notebook instance. Using this method, you can quickly iterate using small datasets to eliminate any structural bugs before you use the container with Amazon SageMaker. We'll walk through local testing later in this notebook.

In this simple application, we only install five files in the container. You may only need that many or, if you have many supporting routines, you may wish to install more. These five show the standard structure of our Python containers, although you are free to choose a different toolset and therefore could have a different layout. If you're writing in a different programming language, you'll certainly have a different layout depending on the frameworks and tools you choose.

The files that we'll put in the container are:

* __`nginx.conf`__ is the configuration file for the nginx front-end. Generally, you should be able to take this file as-is.
* __`predictor.py`__ is the program that actually implements the Flask web server and the decision tree predictions for this app. You'll want to customize the actual prediction parts to your application. Since this algorithm is simple, we do all the processing here in this file, but you may choose to have separate files for implementing your custom logic.
* __`serve`__ is the program started when the container is started for hosting. It simply launches the gunicorn server which runs multiple instances of the Flask app defined in `predictor.py`. You should be able to take this file as-is.
* __`train`__ is the program that is invoked when the container is run for training. You will modify this program to implement your training algorithm.
* __`wsgi.py`__ is a small wrapper used to invoke the Flask app. You should be able to take this file as-is.

In summary, the two files you will probably want to change for your application are `train` and `predictor.py`.

### The Dockerfile

The Dockerfile describes the image that we want to build. You can think of it as describing the complete operating system installation of the system that you want to run. A Docker container running is quite a bit lighter than a full operating system, however, because it takes advantage of Linux on the host machine for the basic operations. 

For the Python science stack, we will start from a standard Ubuntu installation and run the normal tools to install the things needed by scikit-learn. Finally, we add the code that implements our specific algorithm to the container and set up the right environment to run under.

Along the way, we clean up extra space. This makes the container smaller and faster to start.

Let's look at the Dockerfile for the example:

In [11]:
!cat container/Dockerfile

# Build an image that can do training and inference in SageMaker
# This is a Python 2 image that uses the nginx, gunicorn, flask stack
# for serving inferences in a stable way.

FROM ubuntu:16.04

MAINTAINER Amazon AI <sage-learner@amazon.com>


RUN apt-get -y update && apt-get install -y --no-install-recommends \
         wget \
         python \
         nginx \
         ca-certificates \
    && rm -rf /var/lib/apt/lists/*

# Here we get all python packages.
# There's substantial overlap between scipy and numpy that we eliminate by
# linking them together. Likewise, pip leaves the install caches populated which uses
# a significant amount of space. These optimizations save a fair amount of space in the
# image, which reduces start up time.
RUN wget https://bootstrap.pypa.io/get-pip.py && python get-pip.py && \
    pip install numpy scipy scikit-learn pandas flask gevent gunicorn h2o sklearn&& \
        (cd /usr/local/lib/python2.7/dist-packages/scipy/.libs; rm *; ln ../../numpy/.libs

### Building and registering the container

The following shell code shows how to build the container image using `docker build` and push the container image to ECR using `docker push`. This code is also available as the shell script `container/build-and-push.sh`, which you can run as `build-and-push.sh decision_trees_sample` to build the image `decision_trees_sample`. 

This code looks for an ECR repository in the account you're using and the current default region (if you're using a SageMaker notebook instance, this will be the region where the notebook instance was created). If the repository doesn't exist, the script will create it.

In [27]:
%%sh

# The name of our algorithm
algorithm_name=decision-trees-sample

cd container

chmod +x decision_trees/train
chmod +x decision_trees/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Login Succeeded
Sending build context to Docker daemon  84.99kB
Step 1/9 : FROM ubuntu:16.04
 ---> 9361ce633ff1
Step 2/9 : MAINTAINER Amazon AI <sage-learner@amazon.com>
 ---> Using cache
 ---> 1c10096bb505
Step 3/9 : RUN apt-get -y update && apt-get install -y --no-install-recommends          wget          python          nginx          ca-certificates     && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 8c2e6f6d342e
Step 4/9 : RUN wget https://bootstrap.pypa.io/get-pip.py && python get-pip.py &&     pip install numpy scipy scikit-learn pandas flask gevent gunicorn h2o sklearn&&         (cd /usr/local/lib/python2.7/dist-packages/scipy/.libs; rm *; ln ../../numpy/.libs/* .) &&         rm -rf /root/.cache
 ---> Using cache
 ---> 294c39e30b0c
Step 5/9 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> a2e64826fb78
Step 6/9 : ENV PYTHONDONTWRITEBYTECODE=TRUE
 ---> Using cache
 ---> cafff5f336f4
Step 7/9 : ENV PATH="/opt/program:${PATH}"
 ---> Using cache
 ---> 70005a1fa79e
Step 8/9

WARNING! Using --password via the CLI is insecure. Use --password-stdin.


## Testing your algorithm on your local machine or on an Amazon SageMaker notebook instance

While you're first packaging an algorithm use with Amazon SageMaker, you probably want to test it yourself to make sure it's working right. In the directory `container/local_test`, there is a framework for doing this. It includes three shell scripts for running and using the container and a directory structure that mimics the one outlined above.

The scripts are:

* `train_local.sh`: Run this with the name of the image and it will run training on the local tree. You'll want to modify the directory `test_dir/input/data/...` to be set up with the correct channels and data for your algorithm. Also, you'll want to modify the file `input/config/hyperparameters.json` to have the hyperparameter settings that you want to test (as strings).
* `serve_local.sh`: Run this with the name of the image once you've trained the model and it should serve the model. It will run and wait for requests. Simply use the keyboard interrupt to stop it.
* `predict.sh`: Run this with the name of a payload file and (optionally) the HTTP content type you want. The content type will default to `text/csv`. For example, you can run `$ ./predict.sh payload.csv text/csv`.

The directories as shipped are set up to test the decision trees sample algorithm presented here.

# Part 2: Training and Hosting your Algorithm in Amazon SageMaker

Once you have your container packaged, you can use it to train and serve models. Let's do that with the algorithm we made above.

## Set up the environment

Here we specify a bucket to use and the role that will be used for working with SageMaker.

In [170]:
# S3 prefix
prefix = 'dataset-h2o'

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

## Create the session

The session remembers our connection parameters to SageMaker. We'll use it to perform all of our SageMaker operations.

In [60]:
import sagemaker as sage
from time import gmtime, strftime
import sagemaker
sess = sage.Session()

## Upload the data for training

When training large models with huge amounts of data, you'll typically use big data tools, like Amazon Athena, AWS Glue, or Amazon EMR, to create your data in S3. For the purposes of this example, we're using some the classic [Iris dataset](https://en.wikipedia.org/wiki/Iris_flower_data_set), which we have included. 

We can use use the tools provided by the SageMaker Python SDK to upload the data to a default bucket. 

In [61]:
WORK_DIRECTORY = 'data'

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)
data_location

INFO:sagemaker:Created S3 bucket: sagemaker-us-east-1-685194443204


's3://sagemaker-us-east-1-685194443204/dataset-h2o'

In [62]:
train_data = sagemaker.session.s3_input(data_location, s3_data_type='S3Prefix', distribution = 'ShardedByS3Key')
data_channels = {'training': train_data }
data_channels

{'training': <sagemaker.session.s3_input at 0x7fafc4cb5f60>}

## Create an estimator and fit the model

In order to use SageMaker to fit our algorithm, we'll create an `Estimator` that defines how to use the container to train. This includes the configuration we need to invoke SageMaker training:

* The __container name__. This is constructed as in the shell commands above.
* The __role__. As defined above.
* The __instance count__ which is the number of machines to use for training.
* The __instance type__ which is the type of machine to use for training.
* The __output path__ determines where the model artifact will be written.
* The __session__ is the SageMaker session object that we defined above.

Then we use fit() on the estimator to train against the data that we uploaded above.

## Single Instance Training

In [ ]:
import time
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/h2o:latest'.format(account, region)

starttime = time.time()
tree = sage.estimator.Estimator(image,
                       role, 1, 'ml.c5.2xlarge',
                       hyperparameters = {'epochs': 100},
                       output_path="s3://{}/output".format(sess.default_bucket()),
                       sagemaker_session=sess)

tree.fit(data_channels)
print('That took {} seconds'.format(time.time() - starttime))

INFO:sagemaker:Creating training-job with name: h2o-2019-04-16-09-08-35-529


2019-04-16 09:08:35 Starting - Starting the training job...
2019-04-16 09:08:37 Starting - Launching requested ML instances.........
2019-04-16 09:10:12 Starting - Preparing the instances for training...
2019-04-16 09:10:57 Downloading - Downloading input data
2019-04-16 09:10:57 Training - Downloading the training image..
['customer10', 'customer14', 'customer13', 'customer71', 'customer75', 'customer30', 'customer21', 'customer5', 'customer44', 'customer61', 'customer87', 'customer7', 'customer23', 'customer31', 'customer67', 'customer2', 'customer90', 'customer80', 'customer41', 'customer27', 'customer86', 'customer11', 'customer53', 'customer99', 'customer58', 'customer8', 'customer85', 'customer33', 'customer69', 'customer66', 'customer39', 'customer45', 'customer3', 'customer37', 'customer52', 'customer42', 'customer19', 'customer51', 'customer82', 'customer77', 'customer81', 'customer25', 'customer94', 'customer12', 'customer32', 'customer38', 'customer55', 'customer34', 'custom

## Distributed Training Over 5 Instances

In [ ]:
import time
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/h2o:latest'.format(account, region)

starttime = time.time()
h2o_model = sage.estimator.Estimator(image,
                       role, 10, 'ml.c5.2xlarge',
                       output_path="s3://{}/output".format(sess.default_bucket()),
                       sagemaker_session=sess)


#Start the training process
h2o_model.fit(data_channels)
print('That took {} seconds'.format(time.time() - starttime))

INFO:sagemaker:Creating training-job with name: h2o-2019-04-16-23-00-41-622


2019-04-16 23:00:41 Starting - Starting the training job...
2019-04-16 23:00:51 Starting - Launching requested ML instances......
2019-04-16 23:02:00 Starting - Preparing the instances for training......
2019-04-16 23:03:05 Downloading - Downloading input data...
2019-04-16 23:03:40 Training - Training image download completed. Training in progress.
['customer2', 'customer55', 'customer28', 'customer82', 'customer100', 'customer91', 'customer64', 'customer37', 'customer46', 'customer73']
Starting the training.
Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_191"; OpenJDK Runtime Environment (build 1.8.0_191-8u191-b12-2ubuntu0.16.04.1-b12); OpenJDK 64-Bit Server VM (build 25.191-b12, mixed mode)
  Starting server from /usr/local/lib/python2.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmphoj7iS
  JVM stdout: /tmp/tmphoj7iS/h2o_unknownUser_started_from

## Cost of Training:

c5.xlarge instance training: $\$$0.662/hour = \$0.00017/second

$\$$0.00017 * 5617 Billable Seconds = \$0.954 Total cost of training on 100 customers.

## Deploy the model

Deploying the model to SageMaker hosting just requires a `deploy` call on the fitted model. This call takes an instance count, instance type, and optionally serializer and deserializer functions. These are used when the resulting predictor is created on the endpoint.

In [281]:
from sagemaker.predictor import csv_serializer
predictor = h2o_model.deploy(1, 'ml.c5.xlarge', serializer= csv_serializer,  endpoint_name='h2o-model-endpoint')

INFO:sagemaker:Creating model with name: h2o-2019-04-18-03-28-24-486
INFO:sagemaker:Creating endpoint with name h2o-model-endpoint


---------------------------------------------------------------!

## Choose some data and use it for a prediction

In order to do some predictions, we'll extract some of the data we used for training and do predictions against it. This is, of course, bad statistical practice, but a good way to see how the mechanism works.

## Prepare Inference Data


In [278]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler,RobustScaler,Normalizer
import h2o
student=pd.read_csv("payload2.csv", sep=',')
student.isnull().any()
student=student.fillna(0)
predictors=list(range(0,15))
cols_to_transform = [ 'continue_drop','gender','caste','guardian','internet' ]
student = pd.get_dummies( student,columns = cols_to_transform )
student = student.drop('student_id', 1)
# Copy the original dataset
scaled_features = student.copy()

# Extract column names to be standardized
col_names = ['mathematics_marks','english_marks','science_marks',
             'science_teacher','languages_teacher','school_id',
             'total_students','total_toilets','establishment_year']

# Standardize the columns and re-assingn to original dataframe
features = scaled_features[col_names]
scaler = RobustScaler().fit_transform(features.values)
scaled_features = pd.DataFrame(scaler, index=student.index, columns=col_names)

preds = scaled_features
preds

,mathematics_marks,english_marks,science_marks,science_teacher,languages_teacher,school_id,total_students,total_toilets,establishment_year
0,-0.058824,1.334862,-0.058824,2.285714,-0.333333,0.622449,-0.583867,-0.535714,0.414938
1,0.764706,-0.399083,0.764706,-0.571429,-0.777778,0.418367,-1.213828,-0.535714,-0.912863
2,0.781046,0.004587,0.781046,-0.571429,1.444444,-0.846939,-0.916773,-1.250000,0.663900
3,0.166667,0.233945,0.166667,0.000000,-0.111111,-0.642857,0.000000,2.750000,-0.746888
4,-0.519608,-0.944954,-0.519608,0.000000,1.000000,-0.540816,1.516005,2.821429,-0.580913
5,0.650327,-0.022936,0.650327,2.285714,0.555556,-0.010204,-0.010243,-1.392857,0.597510
6,-0.764706,-0.821101,-0.764706,0.000000,-0.111111,0.010204,0.061460,-0.464286,0.547718
7,0.297386,-0.344037,0.297386,-1.142857,1.000000,-0.765306,0.353393,0.464286,0.033195
8,-0.075163,0.412844,-0.075163,-2.285714,-0.555556,-0.642857,0.000000,2.750000,-0.746888
9,0.944444,3.018349,0.944444,0.000000,0.111111,0.091837,0.865557,-0.535714,0.182573


In [286]:
import pickle, json
#preds = scaled_features[:1].values
payload = pickle.dumps(preds,protocol=2)

starttime = time.time()

response = client.invoke_endpoint(
    EndpointName='h2o-model-endpoint', 
    CustomAttributes='customer11', 
    ContentType='application/python-pickle',
    Body=payload
)
print('That took {} seconds'.format(time.time() - starttime))
result = json.loads(response['Body'].read().decode())
result

That took 0.4907107353210449 seconds


{'0': {'0': 'Reconstruction.MSE',
  '1': '0.020604140323444535',
  '2': '0.01620859377516786',
  '3': '0.01476215301808201',
  '4': '0.015937522514172943',
  '5': '0.01568952916307722',
  '6': '0.020023705256531483',
  '7': '0.016003355966679157',
  '8': '0.015605816948928411',
  '9': '0.018673724916031312',
  '10': '0.016506244629250818',
  '11': '0.016014217392387245',
  '12': '0.016130615989258353',
  '13': '0.016533850804047193',
  '14': '0.01569838510054329',
  '15': '0.01691305799658832',
  '16': '0.01616421508358378'}}

## Load Testing

In [291]:
from multiprocessing.dummy import Pool as ThreadPool

starttime = time.time()
def model_inference(customer):
    response = client.invoke_endpoint(
        EndpointName='h2o-model-endpoint', 
        CustomAttributes= customer, 
        ContentType='application/python-pickle',
        Body=payload
    )
    return json.loads(response['Body'].read().decode())
    
 
pool = ThreadPool(4) 
results = pool.map(model_inference, 'customer1')
print('That took {} seconds'.format(time.time() - starttime))
results

That took 0.5805854797363281 seconds


[{'0': {'0': 'Reconstruction.MSE',
   '1': '0.07560431070467935',
   '2': '0.06813769485992204',
   '3': '0.06707893863658755',
   '4': '0.06815477312248888',
   '5': '0.06880273286662603',
   '6': '0.07348257182909751',
   '7': '0.06957378449300156',
   '8': '0.06728573011347883',
   '9': '0.07052186292938399',
   '10': '0.07102716148636616',
   '11': '0.07066397327027002',
   '12': '0.06850835112297518',
   '13': '0.07106018617982199',
   '14': '0.06915940884224012',
   '15': '0.07035211716615651',
   '16': '0.06913012656943704'}},
 {'0': {'0': 'Reconstruction.MSE',
   '1': '0.0037235365059257995',
   '2': '4.744696133893514E-4',
   '3': '4.34758009239139E-4',
   '4': '3.887129935941903E-4',
   '5': '4.833916634498458E-4',
   '6': '0.0036808206955008206',
   '7': '3.899580347197028E-4',
   '8': '4.385755396520982E-4',
   '9': '0.0037149195139546797',
   '10': '7.830796363760244E-4',
   '11': '3.8135069727798044E-4',
   '12': '4.106585762973111E-4',
   '13': '5.214831684568509E-4',
  